In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq pydantic-ai==0.0.14 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


In [2]:
!gdown 1zXnEv6yiQb39Mb9UQstHGx0vbcWAQHcM

Downloading...
From: https://drive.google.com/uc?id=1zXnEv6yiQb39Mb9UQstHGx0vbcWAQHcM
To: /content/app-reviews.db
100% 73.7k/73.7k [00:00<00:00, 60.2MB/s]


In [3]:
import os
import sqlite3
import textwrap
from dataclasses import dataclass
from pathlib import Path
from typing import List

import nest_asyncio
from google.colab import userdata
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

DB_PATH = Path("./app-reviews.db")
MODEL_NAME = "openai:gpt-4o-mini"

nest_asyncio.apply()

## Get Reviews

In [4]:
@dataclass
class Review:
    package_name: str
    text: str
    rating: int

## Tools

In [5]:
def fetch_reviews(
    min_rating: int,
    max_rating: int,
    max_reviews: int = 30,
    min_words_in_review: int = 8,
) -> List[Review]:
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT package_name, review, rating
            FROM reviews
            WHERE rating >= ?
            AND rating <= ?
            AND review IS NOT NULL
            ORDER BY RANDOM()
""",
            (min_rating, max_rating),
        )
        reviews = []

        for package_name, review, rating in cursor.fetchall():
            word_count = len(review.split())
            if word_count < min_words_in_review:
                continue
            reviews.append(
                Review(package_name=package_name, text=review, rating=rating)
            )
        return reviews[:max_reviews]

In [6]:
reviews = fetch_reviews(2, 4, max_reviews=10)
reviews[0]

Review(package_name='org.isoron.uhabits', text='Please provide option for full day track also Other wise is ok ok app', rating=3)

## Dependencies

In [7]:
@dataclass
class Dependencies:
    app_description: str

## Agents

In [8]:
PRODUCT_IMPROVEMENT_PROMPT = """
You help app developers and product managers identify areas for improvement based
on user feedback.
""".strip()


class ProductImprovementResult(BaseModel):
    issues: List[str] = Field(
        description="Prioritized list of common bugs or issues (e.g., 'app crashes on startup')"
    )
    feature_requests: List[str] = Field(
        description="Suggest top requested features (e.g., 'add dark mode')"
    )


improvement_agent = Agent(
    MODEL_NAME,
    deps_type=Dependencies,
    result_type=ProductImprovementResult,
    system_prompt=PRODUCT_IMPROVEMENT_PROMPT,
    tools=[fetch_reviews],
)

In [9]:
MARKETING_PROMPT = """
You're marketer that focuses on engaging mobile app users.
You have a nag for writing clear and focused marketing copy.
""".strip()


class MarketingResult(BaseModel):
    praised_features: List[str] = Field(
        description="Prioritized list of features that users praise the most. . One item per line."
    )
    important_phrases: List[str] = Field(
        description="List of phrases that the marketing team should include in the copy. One item per line."
    )


marketing_agent = Agent(
    MODEL_NAME,
    deps_type=Dependencies,
    result_type=MarketingResult,
    system_prompt=MARKETING_PROMPT,
    tools=[fetch_reviews],
)

In [10]:
class PlanningResult(BaseModel):
    app_names: List[str] = Field(description="List of suggested app names")
    marketing_copy: str = Field(
        description="Marketing copy of the app to build. 2-3 paragraphs at most"
    )
    mvp_features: List[str] = Field(
        description="List of features to include in the MVP of the app"
    )
    possible_issues: List[str] = Field(
        description="Prioritized list of possible bugs that might come up during development"
    )


planner_agent = Agent(MODEL_NAME, deps_type=Dependencies, result_type=PlanningResult)

In [11]:
@planner_agent.system_prompt
async def get_system_prompt(ctx: RunContext[Dependencies]) -> str:
    return f"""
You're planning an MVP for new {ctx.deps.app_description} app. You'll propose
the best possible way to approach the process of building it.
""".strip()

## Run Agents

In [14]:
deps = Dependencies(
    app_description="Todo/habit tracking app with focus on personal development"
)

In [15]:
%%time
improvement_result = improvement_agent.run_sync(
    "Analyze the reviews for possible improvements", deps=deps
)

CPU times: user 101 ms, sys: 7.42 ms, total: 108 ms
Wall time: 4.54 s


In [16]:
for item in improvement_result.new_messages():
    print(item)

ModelRequest(parts=[SystemPromptPart(content='You help app developers and product managers identify areas for improvement based\non user feedback.', part_kind='system-prompt'), UserPromptPart(content='Analyze the reviews for possible improvements', timestamp=datetime.datetime(2024, 12, 19, 21, 50, 14, 629654, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request')
ModelResponse(parts=[ToolCallPart(tool_name='fetch_reviews', args=ArgsJson(args_json='{"min_rating": 1, "max_rating": 3, "max_reviews": 10, "min_words_in_review": 5}'), tool_call_id='call_C5bXZyYXMCgRcsak8t2WZFrK', part_kind='tool-call'), ToolCallPart(tool_name='fetch_reviews', args=ArgsJson(args_json='{"min_rating": 4, "max_rating": 5, "max_reviews": 10, "min_words_in_review": 5}'), tool_call_id='call_LSUgzzI27MV5x7aAhUNLtJJi', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 19, 21, 50, 15, tzinfo=datetime.timezone.utc), kind='response')
ModelRequest(parts=[ToolReturnPart(tool_name='fetch_rev

In [17]:
for item in improvement_result.data.issues:
    print(f"- {item}")

- Account recovery issues after changing devices
- Difficulty in transferring premium features between devices
- High subscription cost for premium features
- Limited user interaction features with the app
- Lack of notifications for habit tracking


In [18]:
for item in improvement_result.data.feature_requests:
    print(f"- {item}")

- Add more interactive features for user engagement
- Implement grouping of habits for better organization
- Allow users to link Google accounts for data recovery
- Introduce a system for compensating missed daily rewards
- Provide an option to log time for habits


In [19]:
%%time
marketing_result = marketing_agent.run_sync(
    "Prepare marketing materials based on the reviews", deps=deps
)

CPU times: user 53.9 ms, sys: 5.92 ms, total: 59.8 ms
Wall time: 2.9 s


In [20]:
for item in marketing_result.new_messages():
    print(item)

ModelRequest(parts=[SystemPromptPart(content="You're marketer that focuses on engaging mobile app users.\nYou have a nag for writing clear and focused marketing copy.", part_kind='system-prompt'), UserPromptPart(content='Prepare marketing materials based on the reviews', timestamp=datetime.datetime(2024, 12, 19, 21, 51, 21, 630961, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request')
ModelResponse(parts=[ToolCallPart(tool_name='fetch_reviews', args=ArgsJson(args_json='{"min_rating": 4, "max_rating": 5, "max_reviews": 5, "min_words_in_review": 15}'), tool_call_id='call_wu863ELcesZrl2XdSshGQMRx', part_kind='tool-call'), ToolCallPart(tool_name='fetch_reviews', args=ArgsJson(args_json='{"min_rating": 1, "max_rating": 2, "max_reviews": 5, "min_words_in_review": 15}'), tool_call_id='call_vK8cXufNZjWmrfqtaGsBbLdW', part_kind='tool-call')], timestamp=datetime.datetime(2024, 12, 19, 21, 51, 21, tzinfo=datetime.timezone.utc), kind='response')
ModelRequest(parts=[ToolReturnPa

In [21]:
for item in marketing_result.data.praised_features:
    print(f"- {item}")

- Interactive experience
- Self-care focus
- No ads
- Useful for self-help and reminders
- User-friendly design


In [22]:
for item in marketing_result.data.important_phrases:
    print(f"- {item}")

- AMAZING app
- Track your habits
- Make your self-care journey enjoyable
- Simplify your life
- Join a supportive community
- Customizable home styles


In [23]:
%%time

prompt = f"""Propose an MVP based on these reports:

# Improvements:

common issues:
{improvement_result.data.issues}

feature requests:
{improvement_result.data.feature_requests}

# Marketing:

praised features:
{marketing_result.data.praised_features}

important phrases:
{marketing_result.data.important_phrases}
"""

planner_result = planner_agent.run_sync(prompt, deps=deps)

CPU times: user 55.6 ms, sys: 8.25 ms, total: 63.8 ms
Wall time: 8.57 s


In [24]:
for item in planner_result.data.app_names:
    print(item)

HabitHarmony
SelfCare Tracker
GoalGetter
HabitHub
Tracktastic


In [25]:
for item in planner_result.data.marketing_copy.split("\n"):
    print(f"{textwrap.fill(item, 100)}")

Introducing our innovative new app designed to simplify your self-care journey and make habit
tracking engaging and rewarding. With a user-friendly design and interactive experience, you'll love
tracking your habits and making personal development a fun part of your day. Join a supportive
community and celebrate your progress as you journey towards a better you!

Our app is ad-free and comes with customizable home styles to fit your personal preference. Stay
motivated and accountable with our notification systems, reward mechanisms, and unique features that
encourage user interaction. Track your habits with ease and make self-care an enjoyable part of your
life!


In [26]:
for item in planner_result.data.mvp_features:
    print(f"- {item}")

- User-friendly interface
- Habit logging system
- Grouping of habits feature
- Notifications for habit tracking
- Integration with Google accounts for recovery
- Compensation system for missed rewards
- Interactive elements for user engagement


In [27]:
for item in planner_result.data.possible_issues:
    print(f"- {item}")

- Account recovery issues after changing devices
- Difficulty in transferring premium features between devices
- High subscription cost for premium features
- Limited user interaction features with the app
- Lack of notifications for habit tracking


In [28]:
planner_result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You're planning an MVP for new Todo/habit tracking app with focus on personal development app. You'll propose\nthe best possible way to approach the process of building it.", part_kind='system-prompt'), UserPromptPart(content="Propose an MVP based on these reports:\n\n# Improvements:\n\ncommon issues:\n['Account recovery issues after changing devices', 'Difficulty in transferring premium features between devices', 'High subscription cost for premium features', 'Limited user interaction features with the app', 'Lack of notifications for habit tracking']\n\nfeature requests:\n['Add more interactive features for user engagement', 'Implement grouping of habits for better organization', 'Allow users to link Google accounts for data recovery', 'Introduce a system for compensating missed daily rewards', 'Provide an option to log time for habits']\n\n# Marketing:\n\npraised features:\n['Interactive experience', 'Self-care focus', 'No ads', 'Useful

## Chat History

In [31]:
%%time
planner_new_result = planner_agent.run_sync(
    "Focus the mvp features on the app functionality - habit tracking and tasks. Give a detailed explanation of the feature",
    message_history=planner_result.new_messages(),
    deps=deps,
)

CPU times: user 43.6 ms, sys: 2.96 ms, total: 46.6 ms
Wall time: 4.51 s


In [32]:
for item in planner_new_result.data.mvp_features:
    print(f"- {textwrap.fill(item, 100)}")
    print()

- Habit Logging System: Allow users to easily log habits with customizable options like frequency,
duration, and type, ensuring they can track what matters most to them.

- Task Management Features: Introduce an intuitive task list that enables users to add, edit, and
delete tasks with due dates and priority levels to help them stay organized.

- Habit Grouping: Allow users to categorize habits into groups for better organization and management,
helping them maintain focus on different areas of their life.

- Notifications & Reminders: Enable customizable notifications to remind users of their daily habits
and tasks, helping to foster accountability and consistency.

- Integration with Google Accounts: Provide easy account recovery and data synchronization options
with Google accounts so users can seamlessly transition across devices without losing progress.

- Reward System: Implement a system that compensates users for completing daily habits and tasks,
encouraging positive reinforce